# Importing Modules
- Torch
- Pandas
- Numpy
- PIL
- Torchmetrics

In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 811.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import torch
import torchvision
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import torchmetrics


# Downloading Dataset from kaggle
`https://www.kaggle.com/datasets/zhangweiled/lidcidri/data`

In [3]:
# Step 1: Install Kaggle
!pip install kaggle

# Step 2: Upload kaggle.json
from google.colab import files
files.upload()  # Upload your kaggle.json file here

# Step 3: Set up Kaggle API token
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the dataset directly to /content/
!kaggle datasets download -d zhangweiled/lidcidri -p /content

# Step 5: Unzip the dataset to /content/lidcidri
!unzip /content/lidcidri.zip -d /content/lidcidri

# Step 6: Verify the path
path = "/content/lidcidri"
print("Path to dataset files:", path)

Streaming output truncated to the last 5000 lines.
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/images/slice-1.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/images/slice-2.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/images/slice-3.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-0/slice-0.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-0/slice-1.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-0/slice-2.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-0/slice-3.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-1/slice-0.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-1/slice-1.png  
  inflating: /content/lidcidri/LIDC-IDRI-slices/LIDC-IDRI-0947/nodule-3/mask-1/slice-2.png  
  inflating: /conte

# Cleaning CSV file containing mapping

In [13]:
# Loading CSV

data = pd.read_csv("/content/dataset_with_image_links.csv")
data = data[['Cancer_Label', 'Image_Link']]
data = data.dropna()
data.head()

,Cancer_Label,Image_Link
0,0,/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-I...
1,0,/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-I...
2,0,/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-I...
3,0,/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-I...
4,0,/kaggle/input/lidcidri/LIDC-IDRI-slices/LIDC-I...


In [14]:
# replacing image links with relative paths

data['Image_Link'] = data['Image_Link'].str.replace('/kaggle/input/lidcidri/LIDC-IDRI-slices/', '/content/1/LIDC-IDRI-slices/')


In [15]:
data

,Cancer_Label,Image_Link
0,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-0001/nod...
1,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-0002/nod...
2,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-0003/nod...
3,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-0003/nod...
4,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-0003/nod...
...,...,...
2625,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-1011/nod...
2626,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-1011/nod...
2627,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-1011/nod...
2628,0,/content/1/LIDC-IDRI-slices/LIDC-IDRI-1011/nod...


In [16]:
# checking count of each class (2 classes)
data.groupby("Cancer_Label").count()


,Image_Link
Cancer_Label,
0,2367
1,263


In [17]:
# transforming folder images into first level images

def process_image_links(data):
    new_data = []
    for index, row in data.iterrows():
        image_path = row['Image_Link']
        if os.path.isdir(image_path):
            for filename in os.listdir(image_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
                    image_file_path = os.path.join(image_path, filename)
                    new_data.append({'Cancer_Label': row['Cancer_Label'], 'Image_Link': image_file_path})
        else:
            if os.path.exists(image_path): # Check if the file actually exists
              new_data.append({'Cancer_Label': row['Cancer_Label'], 'Image_Link': image_path})
    return pd.DataFrame(new_data)

data = process_image_links(data)
print(data.head())
print(data.groupby("Cancer_Label").count())


Empty DataFrame
Columns: []
Index: []


KeyError: 'Cancer_Label'

In [ ]:
# Balancing classes

def balance_classes(data, samples_per_class=1441):
    balanced_data = []
    for label in data['Cancer_Label'].unique():
        label_data = data[data['Cancer_Label'] == label]
        if len(label_data) > samples_per_class:
            balanced_data.append(label_data.sample(n=samples_per_class, random_state=42))  # Use random_state for reproducibility
        else:
            balanced_data.append(label_data)
    return pd.concat(balanced_data).reset_index(drop=True)

data = balance_classes(data)
print(data.groupby("Cancer_Label").count())



In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os

class LIDCDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.data.iloc[idx, 1]
        image = Image.open(img_path).convert('RGB')
        label = self.data.iloc[idx, 0]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- TRANSFORMS ---
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# --- DATASET SPLIT + LOADERS ---
def get_dataloaders(data, batch_size=32):
    full_dataset = LIDCDataset(data=data, transform=None)

    train_size = int(0.7 * len(full_dataset))  # 70% train
    val_size = int(0.15 * len(full_dataset))   # 15% val
    test_size = len(full_dataset) - train_size - val_size  # 15% test

    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    # Set correct transforms
    train_dataset.dataset.transform = train_transform
    val_dataset.dataset.transform = val_test_transform
    test_dataset.dataset.transform = val_test_transform

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = get_dataloaders(data)

# Training Resnet18

In [ ]:
# --- MODEL ---
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.8),
    nn.Linear(model.fc.in_features, 1)
)  # Binary classification
model = model.to(device)

# --- LOSS + OPTIMIZER ---
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# --- METRICS ---
train_f1 = torchmetrics.classification.BinaryF1Score().to(device)
val_f1 = torchmetrics.classification.BinaryF1Score().to(device)
test_f1 = torchmetrics.classification.BinaryF1Score().to(device)

history_train = []
history_val = []

# --- TRAIN FUNCTION ---
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    global history_train, history_val

    best_val_f1 = 0.0  # Track the best validation F1 score

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_f1.reset()

        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training", leave=False)
        for images, labels in loop:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            train_f1.update(preds_class, labels)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        epoch_f1_score = train_f1.compute().item()

        history_train.append((epoch_loss, epoch_acc, epoch_f1_score))

        # --- Validation ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_f1.reset()

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation", leave=False):
                images = images.to(device)
                labels = labels.float().unsqueeze(1).to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                preds = torch.sigmoid(outputs)
                preds_class = (preds > 0.5).float()

                val_correct += (preds_class == labels).sum().item()
                val_total += labels.size(0)

                val_f1.update(preds_class, labels)

        val_epoch_loss = val_loss / len(val_loader)
        val_epoch_acc = val_correct / val_total
        val_epoch_f1_score = val_f1.compute().item()
        history_val.append((val_epoch_loss, val_epoch_acc, val_epoch_f1_score))

        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Train F1: {epoch_f1_score:.4f} || "
              f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f} | Val F1: {val_epoch_f1_score:.4f}")

        # Save the model if validation F1 score improves
        if val_epoch_f1_score > best_val_f1:
            torch.save(model.state_dict(), 'best_model.pth')
            best_val_f1 = val_epoch_f1_score
            print(f"New best model saved with F1 score: {val_epoch_f1_score:.4f}")

# --- TEST FUNCTION ---
def test(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    test_f1.reset()

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            test_f1.update(preds_class, labels)

    test_epoch_loss = test_loss / len(test_loader)
    test_epoch_acc = correct / total
    test_epoch_f1_score = test_f1.compute().item()

    print(f"Test Results -> Loss: {test_epoch_loss:.4f} | Accuracy: {test_epoch_acc:.4f} | F1 Score: {test_epoch_f1_score:.4f}")

# --- START TRAINING ---
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# --- LOAD BEST MODEL AND START TESTING ---
model.load_state_dict(torch.load('best_model.pth'))
test(model, test_loader)

In [ ]:
import matplotlib.pyplot as plt

# Assuming history_train and history_val are lists of tuples (loss, accuracy, f1_score)
train_losses, train_accuracies, train_f1s = zip(*history_train)
val_losses, val_accuracies, val_f1s = zip(*history_val)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(train_f1s, label='Train F1-Score')
plt.plot(val_f1s, label='Validation F1-Score')
plt.xlabel('Epoch')
plt.ylabel('F1-Score')
plt.title('F1-Score Curves')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:

# --- MODEL ---
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
model = model.to(device)

# --- LOSS + OPTIMIZER ---
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# --- METRICS ---
train_f1 = torchmetrics.classification.BinaryF1Score().to(device)
val_f1 = torchmetrics.classification.BinaryF1Score().to(device)
test_f1 = torchmetrics.classification.BinaryF1Score().to(device)

history_train = []
history_val = []


# --- TRAIN FUNCTION ---
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    global history_train, history_val

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_f1.reset()

        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training", leave=False)
        for images, labels in loop:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            train_f1.update(preds_class, labels)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        epoch_f1_score = train_f1.compute().item()

        history_train.append((epoch_loss, epoch_acc, epoch_f1_score))
        # --- Validation ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_f1.reset()

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation", leave=False):
                images = images.to(device)
                labels = labels.float().unsqueeze(1).to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                preds = torch.sigmoid(outputs)
                preds_class = (preds > 0.5).float()

                val_correct += (preds_class == labels).sum().item()
                val_total += labels.size(0)

                val_f1.update(preds_class, labels)

        val_epoch_loss = val_loss / len(val_loader)
        val_epoch_acc = val_correct / val_total
        val_epoch_f1_score = val_f1.compute().item()
        history_val.append((val_epoch_loss, val_epoch_acc, val_epoch_f1_score))
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Train F1: {epoch_f1_score:.4f} || "
              f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f} | Val F1: {val_epoch_f1_score:.4f}")

# --- TEST FUNCTION ---
def test(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    test_f1.reset()

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            test_f1.update(preds_class, labels)

    test_epoch_loss = test_loss / len(test_loader)
    test_epoch_acc = correct / total
    test_epoch_f1_score = test_f1.compute().item()

    print(f"Test Results -> Loss: {test_epoch_loss:.4f} | Accuracy: {test_epoch_acc:.4f} | F1 Score: {test_epoch_f1_score:.4f}")

# --- START TRAINING ---
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

# --- START TESTING ---
test(model, test_loader)


In [ ]:
import matplotlib.pyplot as plt

# Assuming history_train and history_val are lists of tuples (loss, accuracy, f1_score)
train_losses, train_accuracies, train_f1s = zip(*history_train)
val_losses, val_accuracies, val_f1s = zip(*history_val)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(train_f1s, label='Train F1-Score')
plt.plot(val_f1s, label='Validation F1-Score')
plt.xlabel('Epoch')
plt.ylabel('F1-Score')
plt.title('F1-Score Curves')
plt.legend()

plt.tight_layout()
plt.show()


#  EfficientNet

In [ ]:
model = models.efficientnet_b0(pretrained=True)  # Or efficientnet_b1/b2 if you want larger

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
model = model.to(device)


# --- LOSS + OPTIMIZER ---
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# --- METRICS ---
train_f1 = torchmetrics.classification.BinaryF1Score().to(device)
val_f1 = torchmetrics.classification.BinaryF1Score().to(device)
test_f1 = torchmetrics.classification.BinaryF1Score().to(device)

history_train = []
history_val = []


# --- TRAIN FUNCTION ---
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    global history_train, history_val

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_f1.reset()

        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training", leave=False)
        for images, labels in loop:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            train_f1.update(preds_class, labels)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        epoch_f1_score = train_f1.compute().item()

        history_train.append((epoch_loss, epoch_acc, epoch_f1_score))
        # --- Validation ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_f1.reset()

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation", leave=False):
                images = images.to(device)
                labels = labels.float().unsqueeze(1).to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                preds = torch.sigmoid(outputs)
                preds_class = (preds > 0.5).float()

                val_correct += (preds_class == labels).sum().item()
                val_total += labels.size(0)

                val_f1.update(preds_class, labels)

        val_epoch_loss = val_loss / len(val_loader)
        val_epoch_acc = val_correct / val_total
        val_epoch_f1_score = val_f1.compute().item()
        history_val.append((val_epoch_loss, val_epoch_acc, val_epoch_f1_score))
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Train F1: {epoch_f1_score:.4f} || "
              f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f} | Val F1: {val_epoch_f1_score:.4f}")

# --- TEST FUNCTION ---
def test(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    test_f1.reset()

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            test_f1.update(preds_class, labels)

    test_epoch_loss = test_loss / len(test_loader)
    test_epoch_acc = correct / total
    test_epoch_f1_score = test_f1.compute().item()

    print(f"Test Results -> Loss: {test_epoch_loss:.4f} | Accuracy: {test_epoch_acc:.4f} | F1 Score: {test_epoch_f1_score:.4f}")

# --- START TRAINING ---
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

# --- START TESTING ---
test(model, test_loader)


In [ ]:
import matplotlib.pyplot as plt

# Assuming history_train and history_val are lists of tuples (loss, accuracy, f1_score)
train_losses, train_accuracies, train_f1s = zip(*history_train)
val_losses, val_accuracies, val_f1s = zip(*history_val)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(train_f1s, label='Train F1-Score')
plt.plot(val_f1s, label='Validation F1-Score')
plt.xlabel('Epoch')
plt.ylabel('F1-Score')
plt.title('F1-Score Curves')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
model = models.efficientnet_b0(pretrained=True)  # Or efficientnet_b1/b2 if you want larger
# Replace classifier
model.classifier[1] = nn.Sequential(
    nn.Dropout(0.8),
    nn.Linear(model.classifier[1].in_features, 1)
)
model = model.to(device)


# --- LOSS + OPTIMIZER ---
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# --- METRICS ---
train_f1 = torchmetrics.classification.BinaryF1Score().to(device)
val_f1 = torchmetrics.classification.BinaryF1Score().to(device)
test_f1 = torchmetrics.classification.BinaryF1Score().to(device)

history_train = []
history_val = []


# --- TRAIN FUNCTION ---
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    global history_train, history_val

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_f1.reset()

        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training", leave=False)
        for images, labels in loop:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            train_f1.update(preds_class, labels)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        epoch_f1_score = train_f1.compute().item()

        history_train.append((epoch_loss, epoch_acc, epoch_f1_score))
        # --- Validation ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_f1.reset()

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation", leave=False):
                images = images.to(device)
                labels = labels.float().unsqueeze(1).to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                preds = torch.sigmoid(outputs)
                preds_class = (preds > 0.5).float()

                val_correct += (preds_class == labels).sum().item()
                val_total += labels.size(0)

                val_f1.update(preds_class, labels)

        val_epoch_loss = val_loss / len(val_loader)
        val_epoch_acc = val_correct / val_total
        val_epoch_f1_score = val_f1.compute().item()
        history_val.append((val_epoch_loss, val_epoch_acc, val_epoch_f1_score))
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Train F1: {epoch_f1_score:.4f} || "
              f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f} | Val F1: {val_epoch_f1_score:.4f}")

# --- TEST FUNCTION ---
def test(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    test_f1.reset()

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            correct += (preds_class == labels).sum().item()
            total += labels.size(0)

            test_f1.update(preds_class, labels)

    test_epoch_loss = test_loss / len(test_loader)
    test_epoch_acc = correct / total
    test_epoch_f1_score = test_f1.compute().item()

    print(f"Test Results -> Loss: {test_epoch_loss:.4f} | Accuracy: {test_epoch_acc:.4f} | F1 Score: {test_epoch_f1_score:.4f}")

# --- START TRAINING ---
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

# --- START TESTING ---
test(model, test_loader)


In [ ]:
import matplotlib.pyplot as plt

# Assuming history_train and history_val are lists of tuples (loss, accuracy, f1_score)
train_losses, train_accuracies, train_f1s = zip(*history_train)
val_losses, val_accuracies, val_f1s = zip(*history_val)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(train_f1s, label='Train F1-Score')
plt.plot(val_f1s, label='Validation F1-Score')
plt.xlabel('Epoch')
plt.ylabel('F1-Score')
plt.title('F1-Score Curves')
plt.legend()

plt.tight_layout()
plt.show()


# Training Vision Transformer B1 Pre-trained

In [ ]:
import timm
from torch.optim.lr_scheduler import CosineAnnealingLR
# --- MODEL: TIMM Vision Transformer ---
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=1)
model = model.to(device)

# --- LOSS + OPTIMIZER ---
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)

# --- METRICS ---
train_f1 = torchmetrics.classification.BinaryF1Score().to(device)
val_f1 = torchmetrics.classification.BinaryF1Score().to(device)
test_f1 = torchmetrics.classification.BinaryF1Score().to(device)

# --- TRAINING LOOP ---
num_epochs = 7
best_val_f1 = 0.0
patience = 5
counter = 0
best_model_path = "best_vit_model.pth"

vit_history_train = []
vit_history_val = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    train_f1.reset()

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training", leave=False)
    for images, labels in loop:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        running_loss += loss.item()
        preds = torch.sigmoid(outputs)
        preds_class = (preds > 0.5).float()

        correct += (preds_class == labels).sum().item()
        total += labels.size(0)
        train_f1.update(preds_class, labels)

    scheduler.step()  # Update learning rate

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    epoch_f1 = train_f1.compute().item()

    vit_history_train.append((epoch_loss, epoch_acc, epoch_f1))
    # --- VALIDATION ---
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_f1.reset()

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation", leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            val_correct += (preds_class == labels).sum().item()
            val_total += labels.size(0)
            val_f1.update(preds_class, labels)

    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_acc = val_correct / val_total
    val_epoch_f1 = val_f1.compute().item()

    # --- Save best model ---
    if val_epoch_f1 > best_val_f1:
        best_val_f1 = val_epoch_f1
        torch.save(model.state_dict(), best_model_path)
        counter = 0
    else:
        counter += 1

    # --- Early stopping ---
    if counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

    vit_history_val.append((val_epoch_loss, val_epoch_acc, val_epoch_f1))
    # --- TESTING ---
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    test_f1.reset()

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Testing", leave=False):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds_class = (preds > 0.5).float()

            test_correct += (preds_class == labels).sum().item()
            test_total += labels.size(0)
            test_f1.update(preds_class, labels)

    test_epoch_loss = test_loss / len(test_loader)
    test_epoch_acc = test_correct / test_total
    test_epoch_f1 = test_f1.compute().item()

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Train F1: {epoch_f1:.4f} || "
          f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f} | Val F1: {val_epoch_f1:.4f} || "
          f"Test Loss: {test_epoch_loss:.4f} | Test Acc: {test_epoch_acc:.4f} | Test F1: {test_epoch_f1:.4f}")

# --- Load best model for final evaluation ---
model.load_state_dict(torch.load(best_model_path))
print(f"Loaded best model with Validation F1: {best_val_f1:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming history_train and history_val are lists of tuples (loss, accuracy, f1_score)
train_losses, train_accuracies, train_f1s = zip(*vit_history_train)
val_losses, val_accuracies, val_f1s = zip(*vit_history_val)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(train_f1s, label='Train F1-Score')
plt.plot(val_f1s, label='Validation F1-Score')
plt.xlabel('Epoch')
plt.ylabel('F1-Score')
plt.title('F1-Score Curves')
plt.legend()

plt.tight_layout()
plt.show()
